# 0. 라이브러리

In [1]:
# !pip install -r requirements.txt
!pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=0bc4a72827c26854c3acc9473651a1f10eb53cbf6d2f6484f814dc5977e4571c
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention


# 1. Load Data

In [2]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_self_attention import SeqSelfAttention
from sklearn.preprocessing import MinMaxScaler
from keras.utils.vis_utils import plot_model

from sklearn.metrics import mean_absolute_error, mean_squared_error

import pickle
import random

In [ ]:
# from tensorflow.python.client import device_lib
# tf.config.list_physical_devices('GPU')

In [3]:

seed_num = 42
tf.random.set_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)

In [ ]:
# class config:
#     seed = 42
#     device = "cuda:0"

#     lr = 1e-3
#     epochs = 25
#     batch_size = 32
#     num_workers = 4
#     train_5_folds = True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# data load
with open('/content/drive/MyDrive/00.multi_emtions/03.models/movies_train.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('/content/drive/MyDrive/00.multi_emtions/03.models/movies_test.pkl', 'rb') as f:
    test_set = pickle.load(f)
with open('/content/drive/MyDrive/00.multi_emtions/03.models/movies_validation.pkl', 'rb') as f:
    valid_set = pickle.load(f)


In [ ]:
train_set

,userID,movieID,rating,compound,senti_score,NeMFRating,SBMFRating,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
1633870,143756,31800,1.00,0.5719,3,3,4,0.00,0.00,0.00,0.00,16.67,0.00,0.00,33.33,0.00,16.67
3115182,272600,13746,1.00,0.0000,2,3,3,0.00,11.11,0.00,11.11,0.00,0.00,0.00,11.11,0.00,0.00
2383131,208589,23849,0.50,-0.2023,1,2,3,10.53,5.26,5.26,5.26,0.00,5.26,5.26,5.26,10.53,10.53
2079042,182422,14736,0.75,0.9666,3,3,5,5.04,4.20,9.24,8.40,6.72,7.56,5.04,4.20,14.29,13.45
651398,56973,53386,1.00,0.9260,3,3,5,0.00,27.27,0.00,4.55,18.18,0.00,13.64,40.91,0.00,27.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158490,13844,27705,1.00,0.7351,3,3,4,0.00,0.00,0.00,0.00,8.00,0.00,0.00,20.00,0.00,12.00
1734117,152410,28106,0.75,0.8750,3,3,5,0.00,3.77,0.00,3.77,3.77,0.00,3.77,11.32,3.77,5.66
1564450,137584,43164,1.00,0.6249,3,3,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,33.33
212205,18521,42215,1.00,0.4588,3,3,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
overall = train_set.shape[0] + valid_set.shape[0] + test_set.shape[0]
print('train:validation:test = %.4f:%.4f:%.4f'%((train_set.shape[0] / overall), (valid_set.shape[0] / overall), (test_set.shape[0] / overall)))

train:validation:test = 0.7000:0.1000:0.2000


In [6]:
def norm(x):
    _max = x.max()
    _min = x.min()
    _denom = _max - _min
    return (x - _min) / _denom

train_set['rating'] = norm(train_set['rating'])
valid_set['rating'] = norm(valid_set['rating'])

# 3.Model

In [7]:
def conv_block(n_layers, latent_dims, names):
    ModuleList, DropoutList = [], []
    for i, _ in enumerate(range(n_layers)):
        ModuleList.append(Dense(latent_dims, activation='relu', name=f'{names}{i+1}'))
        latent_dims //= 2
    return ModuleList

# 00. Basic


In [8]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    # Sentiment
    # pos_input = Input(shape=(1,), dtype='int32', name='PositiveInput')
    # pos_dense = Dense(units=1, activation='relu', name='Dense_Positive')(pos_input)

    # neg_input = Input(shape=(1,), dtype='int32', name='NegativeInput')
    # neg_dense = Dense(units=1, activation='relu', name='Dense_Negative')(neg_input)

    # anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    # anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    # disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    # disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    # fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    # fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    # joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    # joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    # sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    # sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    # surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    # surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    # trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    # trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)

    # Sentiment = Concatenate(name='SentimentLayer')([anger_dense, anticipation_dense, disgust_dense, fear_dense, joy_dense, sadness_dense, surprise_dense, trust_dense])
    # Sentiment concatenation
    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I])

    # Final concatenation
    # MLP_U_I_S = Concatenate(name='FinalLayer')([MLP_U_I, Sentiment])

    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input], outputs=outputs)
    return model

In [ ]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'] ],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'] ],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [ ]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [32]
total_layer_dims =  [128, 64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험1

In [ ]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 26s 10ms/step - loss: 0.4316 - val_loss: 0.4113
Epoch 2/100
2199/2199 [==============================] - 11s 5ms/step - loss: 0.3815 - val_loss: 0.4119
Epoch 3/100
2199/2199 [==============================] - 11s 5ms/step - loss: 0.3477 - val_loss: 0.4302
Epoch 4/100
2199/2199 [==============================] - 11s 5ms/step - loss: 0.3020 - val_loss: 0.4730
Epoch 4: early stopping
20099/20099 [==============================] - 28s 1ms/step
embedding:32, layer_dim:128,lr:0.001
MAE: 0.689
RMSE: 0.969
Epoch 1/100
2199/2199 [==============================] - 25s 10ms/step - loss: 0.4301 - val_loss: 0.4123
Epoch 2/100
2199/2199 [==============================] - 11s 5ms/step - loss: 0.3802 - val_loss: 0.4127
Epoch 3/100
2199/2199 [==============================] - 11s 5ms/step - loss: 0.3497 - val_loss: 0.4285
Epoch 4/100
2199/2199 [==============================] - 11s 5ms/step - loss: 0.3142 - val_loss: 0.4788
Epoch 4: early stoppin

In [ ]:
basicMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
basicMLP

,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,32,128,0.001,0.688642,0.968598
1,32,128,0.005,0.704496,0.971354
2,32,64,0.001,0.663203,0.962783
3,32,64,0.005,0.688848,0.966940
4,32,32,0.001,0.673025,0.967684
5,32,32,0.005,0.655941,0.964421


In [ ]:
basicMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.Basic_nonEmotion.csv",index=False)


# 7 emotions

## 1.fear 제외

In [ ]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    # Sentiment
    # pos_input = Input(shape=(1,), dtype='int32', name='PositiveInput')
    # pos_dense = Dense(units=1, activation='relu', name='Dense_Positive')(pos_input)

    # neg_input = Input(shape=(1,), dtype='int32', name='NegativeInput')
    # neg_dense = Dense(units=1, activation='relu', name='Dense_Negative')(neg_input)

    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    # fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    # fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)

    # Sentiment = Concatenate(name='SentimentLayer')([anger_dense, anticipation_dense, disgust_dense, fear_dense, joy_dense, sadness_dense, surprise_dense, trust_dense])
    # Sentiment concatenation
    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, anger_dense, joy_dense, sadness_dense, disgust_dense, anticipation_dense, trust_dense, surprise_dense])

    # Final concatenation
    # MLP_U_I_S = Concatenate(name='FinalLayer')([MLP_U_I, Sentiment])

    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, anger_input, joy_input, sadness_input, disgust_input, anticipation_input, trust_input, surprise_input], outputs=outputs)
    return model

In [ ]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['joy'] , train_set['sadness'] , train_set['disgust'], train_set['anticipation'] , train_set['anger'] , train_set['trust'] , train_set['surprise']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['joy'] ,valid_set['sadness'] ,valid_set['disgust'], valid_set['anticipation'], valid_set['anger'], valid_set['trust'], valid_set['surprise']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['joy'], test_set['sadness'], test_set['disgust'], test_set['anticipation'], test_set['anger'], test_set['trust'], test_set['surprise']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [ ]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [128,64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험fear

In [ ]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 36s 14ms/step - loss: 0.4249 - val_loss: 0.3986
Epoch 2/100
2199/2199 [==============================] - 18s 8ms/step - loss: 0.3662 - val_loss: 0.4003
Epoch 3/100
2199/2199 [==============================] - 18s 8ms/step - loss: 0.3325 - val_loss: 0.4148
Epoch 4/100
2199/2199 [==============================] - 18s 8ms/step - loss: 0.3003 - val_loss: 0.4448
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:128, layer_dim:128,lr:0.001
MAE: 0.659
RMSE: 0.933
Epoch 1/100
2199/2199 [==============================] - 36s 14ms/step - loss: 0.4179 - val_loss: 0.4014
Epoch 2/100
2199/2199 [==============================] - 19s 8ms/step - loss: 0.3659 - val_loss: 0.4011
Epoch 3/100
2199/2199 [==============================] - 19s 8ms/step - loss: 0.3336 - val_loss: 0.4216
Epoch 4/100
2199/2199 [==============================] - 18s 8ms/step - loss: 0.3029 - val_loss: 0.4424
Epoch 5/100
2199/2199

In [ ]:
seven_emotionMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_emotionMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,128,128,0.001,0.658625,0.932586
1,128,128,0.005,0.629360,0.934062
2,128,64,0.001,0.626677,0.929023
3,128,64,0.005,0.667670,0.935061
4,128,32,0.001,0.647487,0.937776
5,128,32,0.005,0.630197,0.935136
6,64,128,0.001,0.622817,0.928172
7,64,128,0.005,0.676776,0.938546
8,64,64,0.001,0.627203,0.930585
9,64,64,0.005,0.630471,0.930675


In [ ]:
seven_emotionMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_fear.csv",index=False)

## 2.joy 제외

In [10]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    # Sentiment
    # pos_input = Input(shape=(1,), dtype='int32', name='PositiveInput')
    # pos_dense = Dense(units=1, activation='relu', name='Dense_Positive')(pos_input)

    # neg_input = Input(shape=(1,), dtype='int32', name='NegativeInput')
    # neg_dense = Dense(units=1, activation='relu', name='Dense_Negative')(neg_input)

    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    # joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    # joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)

    # Sentiment = Concatenate(name='SentimentLayer')([anger_dense, anticipation_dense, disgust_dense, fear_dense, joy_dense, sadness_dense, surprise_dense, trust_dense])
    # Sentiment concatenation
    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, anger_dense, fear_dense, sadness_dense, disgust_dense, anticipation_dense, trust_dense, surprise_dense])

    # Final concatenation
    # MLP_U_I_S = Concatenate(name='FinalLayer')([MLP_U_I, Sentiment])

    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, anger_input, fear_input, sadness_input, disgust_input, anticipation_input, trust_input, surprise_input], outputs=outputs)
    return model

In [9]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['sadness'] , train_set['disgust'], train_set['anticipation'] , train_set['anger'] , train_set['trust'] , train_set['surprise']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['sadness'] ,valid_set['disgust'], valid_set['anticipation'], valid_set['anger'], valid_set['trust'], valid_set['surprise']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['sadness'], test_set['disgust'], test_set['anticipation'], test_set['anger'], test_set['trust'], test_set['surprise']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [11]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 joy

In [12]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 45s 15ms/step - loss: 0.4249 - val_loss: 0.4049
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3747 - val_loss: 0.4059
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3433 - val_loss: 0.4193
Epoch 4/100
2199/2199 [==============================] - 16s 8ms/step - loss: 0.3079 - val_loss: 0.4519
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.672
RMSE: 0.950
Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4223 - val_loss: 0.4057
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3716 - val_loss: 0.4058
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3396 - val_loss: 0.4198
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3096 - val_loss: 0.4613
Epoch 4: early stoppin

In [13]:
seven_joyMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_joyMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.671518,0.949693
1,64,128,0.005,0.687345,0.951520
2,64,64,0.001,0.643172,0.944239
3,64,64,0.005,0.679133,0.952969
4,64,32,0.001,0.660448,0.951849
5,64,32,0.005,0.643288,0.949172
6,32,128,0.001,0.674678,0.951493
7,32,128,0.005,0.642555,0.945569
8,32,64,0.001,0.645691,0.943947
9,32,64,0.005,0.675976,0.950135


In [14]:
seven_joyMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_joy.csv",index=False)

## 3.anger 제외

In [15]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    # anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    # anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)


    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, joy_dense, fear_dense, sadness_dense, disgust_dense, anticipation_dense, trust_dense, surprise_dense])


    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, joy_input, fear_input, sadness_input, disgust_input, anticipation_input, trust_input, surprise_input], outputs=outputs)
    return model

In [16]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['sadness'] , train_set['disgust'], train_set['anticipation'] , train_set['joy'] , train_set['trust'] , train_set['surprise']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['sadness'] ,valid_set['disgust'], valid_set['anticipation'], valid_set['joy'], valid_set['trust'], valid_set['surprise']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['sadness'], test_set['disgust'], test_set['anticipation'], test_set['joy'], test_set['trust'], test_set['surprise']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [17]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 anger

In [18]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4166 - val_loss: 0.3984
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3644 - val_loss: 0.4006
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3316 - val_loss: 0.4131
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.2991 - val_loss: 0.4461
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.657
RMSE: 0.933
Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4153 - val_loss: 0.4022
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3643 - val_loss: 0.3999
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3323 - val_loss: 0.4198
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3045 - val_loss: 0.4503
Epoch 5/100
2199/2199 

In [19]:
seven_angerMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_angerMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.656833,0.932954
1,64,128,0.005,0.630070,0.932843
2,64,64,0.001,0.632520,0.934704
3,64,64,0.005,0.625668,0.933552
4,64,32,0.001,0.645749,0.939629
5,64,32,0.005,0.633482,0.936142
6,32,128,0.001,0.662650,0.935473
7,32,128,0.005,0.626609,0.932826
8,32,64,0.001,0.634162,0.933982
9,32,64,0.005,0.627326,0.931959


In [20]:
seven_angerMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_anger.csv",index=False)

## 4.anticipation 제외

In [21]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    # anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    # anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)


    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, joy_dense, fear_dense, sadness_dense, disgust_dense, anger_dense, trust_dense, surprise_dense])


    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, joy_input, fear_input, sadness_input, disgust_input, anger_input, trust_input, surprise_input], outputs=outputs)
    return model

In [22]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['sadness'] , train_set['disgust'], train_set['anger'] , train_set['joy'] , train_set['trust'] , train_set['surprise']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['sadness'] ,valid_set['disgust'], valid_set['anger'], valid_set['joy'], valid_set['trust'], valid_set['surprise']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['sadness'], test_set['disgust'], test_set['anger'], test_set['joy'], test_set['trust'], test_set['surprise']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [23]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 anticipation

In [24]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 35s 14ms/step - loss: 0.4166 - val_loss: 0.3984
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3645 - val_loss: 0.4004
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3317 - val_loss: 0.4139
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.2991 - val_loss: 0.4437
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.657
RMSE: 0.933
Epoch 1/100
2199/2199 [==============================] - 34s 13ms/step - loss: 0.4153 - val_loss: 0.4012
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3643 - val_loss: 0.3995
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3323 - val_loss: 0.4168
Epoch 4/100
2199/2199 [==============================] - 16s 7ms/step - loss: 0.3040 - val_loss: 0.4386
Epoch 5/100
2199/2199 

In [25]:
seven_antiMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_antiMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.656938,0.932853
1,64,128,0.005,0.629448,0.932382
2,64,64,0.001,0.632833,0.934182
3,64,64,0.005,0.630091,0.932768
4,64,32,0.001,0.645833,0.938380
5,64,32,0.005,0.634961,0.937306
6,32,128,0.001,0.663010,0.935521
7,32,128,0.005,0.627167,0.932687
8,32,64,0.001,0.633773,0.933912
9,32,64,0.005,0.674802,0.937850


In [26]:
seven_antiMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_anti.csv",index=False)

## 5.disgust 제외

In [27]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    # disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    # disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)


    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, joy_dense, fear_dense, sadness_dense, anticipation_dense, anger_dense, trust_dense, surprise_dense])


    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, joy_input, fear_input, sadness_input, anticipation_input, anger_input, trust_input, surprise_input], outputs=outputs)
    return model

In [28]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['sadness'] , train_set['anticipation'], train_set['anger'] , train_set['joy'] , train_set['trust'] , train_set['surprise']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['sadness'] ,valid_set['anticipation'], valid_set['anger'], valid_set['joy'], valid_set['trust'], valid_set['surprise']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['sadness'], test_set['anticipation'], test_set['anger'], test_set['joy'], test_set['trust'], test_set['surprise']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [29]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 disgust

In [30]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4179 - val_loss: 0.3998
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3664 - val_loss: 0.4006
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3347 - val_loss: 0.4132
Epoch 4/100
2199/2199 [==============================] - 16s 7ms/step - loss: 0.3040 - val_loss: 0.4446
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.666
RMSE: 0.936
Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4157 - val_loss: 0.4000
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3636 - val_loss: 0.4010
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3316 - val_loss: 0.4186
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3042 - val_loss: 0.4446
Epoch 4: early stoppin

In [31]:
seven_disgustMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_disgustMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.666492,0.936220
1,64,128,0.005,0.670980,0.937227
2,64,64,0.001,0.631131,0.932572
3,64,64,0.005,0.630290,0.934002
4,64,32,0.001,0.650422,0.941038
5,64,32,0.005,0.628296,0.935072
6,32,128,0.001,0.631276,0.931785
7,32,128,0.005,0.674072,0.938171
8,32,64,0.001,0.635323,0.931958
9,32,64,0.005,0.629468,0.933276


In [32]:
seven_disgustMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_disgust.csv",index=False)

## 6.sadness 제외

In [33]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    # sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    # sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)


    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, joy_dense, fear_dense, disgust_dense, anticipation_dense, anger_dense, trust_dense, surprise_dense])


    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, joy_input, fear_input, disgust_input, anticipation_input, anger_input, trust_input, surprise_input], outputs=outputs)
    return model

In [34]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['disgust'] , train_set['anticipation'], train_set['anger'] , train_set['joy'] , train_set['trust'] , train_set['surprise']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['disgust'] ,valid_set['anticipation'], valid_set['anger'], valid_set['joy'], valid_set['trust'], valid_set['surprise']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['disgust'], test_set['anticipation'], test_set['anger'], test_set['joy'], test_set['trust'], test_set['surprise']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [35]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 sadness

In [36]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4171 - val_loss: 0.3991
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3662 - val_loss: 0.3996
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3351 - val_loss: 0.4109
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3050 - val_loss: 0.4397
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.666
RMSE: 0.935
Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4153 - val_loss: 0.4002
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3635 - val_loss: 0.4005
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3312 - val_loss: 0.4192
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3038 - val_loss: 0.4439
Epoch 4: early stoppin

In [37]:
seven_sadnessMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_sadnessMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.666089,0.935067
1,64,128,0.005,0.672058,0.939406
2,64,64,0.001,0.632860,0.933772
3,64,64,0.005,0.676487,0.939807
4,64,32,0.001,0.647666,0.940679
5,64,32,0.005,0.637734,0.937232
6,32,128,0.001,0.627912,0.930073
7,32,128,0.005,0.672953,0.937660
8,32,64,0.001,0.634974,0.933200
9,32,64,0.005,0.674353,0.939175


In [38]:
seven_sadnessMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_sadness.csv",index=False)

## 7.surprise 제외

In [39]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    # surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    # surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)


    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, joy_dense, fear_dense, disgust_dense, anticipation_dense, anger_dense, trust_dense, sadness_dense])


    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, joy_input, fear_input, disgust_input, anticipation_input, anger_input, trust_input, sadness_input], outputs=outputs)
    return model

In [40]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['disgust'] , train_set['anticipation'], train_set['anger'] , train_set['joy'] , train_set['trust'] , train_set['sadness']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['disgust'] ,valid_set['anticipation'], valid_set['anger'], valid_set['joy'], valid_set['trust'], valid_set['sadness']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['disgust'], test_set['anticipation'], test_set['anger'], test_set['joy'], test_set['trust'], test_set['sadness']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [41]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 surprise

In [42]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4163 - val_loss: 0.3983
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3657 - val_loss: 0.3991
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3358 - val_loss: 0.4107
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3074 - val_loss: 0.4384
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.662
RMSE: 0.932
Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4140 - val_loss: 0.3999
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3627 - val_loss: 0.3996
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3303 - val_loss: 0.4175
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3028 - val_loss: 0.4430
Epoch 5/100
2199/2199 

In [43]:
seven_surpriseMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_surpriseMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.661892,0.932340
1,64,128,0.005,0.623101,0.929309
2,64,64,0.001,0.627462,0.927580
3,64,64,0.005,0.629067,0.931272
4,64,32,0.001,0.641651,0.935940
5,64,32,0.005,0.627350,0.934102
6,32,128,0.001,0.623085,0.927795
7,32,128,0.005,0.625755,0.929252
8,32,64,0.001,0.628227,0.927666
9,32,64,0.005,0.627212,0.931745


In [44]:
seven_surpriseMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_surprise.csv",index=False)

## 8.trust 제외

In [45]:
def ModelBuild(user_num, item_num, id_dims, total_layer_dims, total_n_layer):

    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, id_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, id_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='ItemFlatten')(item_embedding)


    # GMF
    # GMF = Multiply()([user_embedding, item_embedding])

    # MLP
    MLP_U_I = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])

    for layer in conv_block(total_n_layer, total_layer_dims, 'UserItemMLP'):
        MLP_U_I = layer(MLP_U_I)


    anger_input = Input(shape=(1,), dtype='int32', name='AngerInput')
    anger_dense = Dense(units=1, activation='relu', name='Dense_Anger')(anger_input)

    anticipation_input = Input(shape=(1,), dtype='int32', name='AnticipationInput')
    anticipation_dense = Dense(units=1, activation='relu', name='Dense_Anticipation')(anticipation_input)

    fear_input = Input(shape=(1,), dtype='int32', name='FearInput')
    fear_dense = Dense(units=1, activation='relu', name='Dense_Fear')(fear_input)

    disgust_input = Input(shape=(1,), dtype='int32', name='DisgustInput')
    disgust_dense = Dense(units=1, activation='relu', name='Dense_Disgust')(disgust_input)

    joy_input = Input(shape=(1,), dtype='int32', name='JoyInput')
    joy_dense = Dense(units=1, activation='relu', name='Dense_Joy')(joy_input)

    sadness_input = Input(shape=(1,), dtype='int32', name='SadnessInput')
    sadness_dense = Dense(units=1, activation='relu', name='Dense_Sadness')(sadness_input)

    surprise_input = Input(shape=(1,), dtype='int32', name='SurpriseInput')
    surprise_dense = Dense(units=1, activation='relu', name='Dense_Surprise')(surprise_input)

    # trust_input = Input(shape=(1,), dtype='int32', name='TrustInput')
    # trust_dense = Dense(units=1, activation='relu', name='Dense_Trust')(trust_input)


    MLP_U_I_S = Concatenate(name='UserItemSentimentLayer')([MLP_U_I, joy_dense, fear_dense, disgust_dense, anticipation_dense, anger_dense, surprise_dense, sadness_dense])


    for layer in conv_block(total_n_layer, (total_layer_dims//(2**(total_n_layer-1))+8)//2, 'UserItemSentimentMLP'):
        MLP_U_I_S = layer(MLP_U_I_S)

    outputs = Dense(1, activation='sigmoid', name='outputs')(MLP_U_I_S)
    model = Model(inputs=[user_input, item_input, joy_input, fear_input, disgust_input, anticipation_input, anger_input, surprise_input, sadness_input], outputs=outputs)
    return model

In [46]:
def model_run_param(id_dims, total_layer_dims,total_n_layer,lr):

    # param_embedding, param_layer_dim, param_lr = [],[],[]
    # MAE, RMSE = [], []
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

    model = ModelBuild(user_num = user_num, item_num = item_num, id_dims = id_dims, total_layer_dims = total_layer_dims, total_n_layer = total_n_layer)

    adam = Adam(learning_rate=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 3, restore_best_weights = True)
    # model.summary()

    history = model.fit([train_set['userID'], train_set['movieID'], train_set['fear'] , train_set['disgust'] , train_set['anticipation'], train_set['anger'] , train_set['joy'] , train_set['surprise'] , train_set['sadness']],
                train_set['rating'],
                batch_size = 1024,
                epochs = 100,
                callbacks=[es],
                validation_data = ([valid_set['userID'], valid_set['movieID'],valid_set['fear'] ,valid_set['disgust'] ,valid_set['anticipation'], valid_set['anger'], valid_set['joy'], valid_set['surprise'], valid_set['sadness']],
                valid_set['rating']))

    prediction = model.predict([test_set['userID'], test_set['movieID'], test_set['fear'], test_set['disgust'], test_set['anticipation'], test_set['anger'], test_set['joy'], test_set['surprise'], test_set['sadness']])

    # normalize 안해도 되겠단 생각이..
    # prediction = 4 * norm(prediction) + 1
    prediction = 4 * prediction + 1

    MAE_temp = mean_absolute_error(test_set['rating'], prediction)
    RMSE_temp = mean_squared_error(test_set['rating'], prediction, squared = False)
    embedding_size = id_dims
    layer_dim = total_layer_dims
    lr = lr
    print(f'embedding:{embedding_size}, layer_dim:{layer_dim},lr:{lr}')
    print(f'MAE: {MAE_temp:.3f}')
    print(f'RMSE: {RMSE_temp:.3f}')

    # MAE.append(MAE_temp)
    # RMSE.append(RMSE_temp)
    # param_embedding.append(embedding_size)
    # param_layer_dim.append(layer_dim)
    # param_lr.append(lr)

    return MAE_temp, RMSE_temp, embedding_size, layer_dim, lr

In [47]:
# 다시 시도...
user_num = 281335
item_num = 59044

id_dims = [64,32]
total_layer_dims =  [128,64,32] # ex. 128 -> 128 , 64, 32...
total_n_layer = 3 # concat(64) -> 64-32-16
lr = [0.001, 0.005]

### 실험 trust

In [48]:
MAE, RMSE, param_embedding, param_layer_dim, param_lr = [], [], [], [], []
for a in id_dims:
  for b in total_layer_dims:
    for i in lr:
      MAE_temp, RMSE_temp, embedding_size, layer_dim, learning_rate = model_run_param(a, b,3,i)

      MAE.append(MAE_temp)
      RMSE.append(RMSE_temp)
      param_embedding.append(embedding_size)
      param_layer_dim.append(layer_dim)
      param_lr.append(learning_rate)

Epoch 1/100
2199/2199 [==============================] - 35s 13ms/step - loss: 0.4163 - val_loss: 0.3984
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3659 - val_loss: 0.3988
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3358 - val_loss: 0.4101
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3070 - val_loss: 0.4405
Epoch 4: early stopping
20099/20099 [==============================] - 44s 2ms/step
embedding:64, layer_dim:128,lr:0.001
MAE: 0.662
RMSE: 0.932
Epoch 1/100
2199/2199 [==============================] - 34s 13ms/step - loss: 0.4140 - val_loss: 0.3997
Epoch 2/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3627 - val_loss: 0.3998
Epoch 3/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3307 - val_loss: 0.4173
Epoch 4/100
2199/2199 [==============================] - 17s 8ms/step - loss: 0.3029 - val_loss: 0.4419
Epoch 4: early stoppin

In [49]:
seven_trustMLP = pd.DataFrame(zip(param_embedding, param_layer_dim, param_lr, MAE, RMSE), columns=['embedding size','layer_dim','Learning Rate','MAE','RMSE'])
seven_trustMLP



,embedding size,layer_dim,Learning Rate,MAE,RMSE
0,64,128,0.001,0.662099,0.932261
1,64,128,0.005,0.673252,0.936616
2,64,64,0.001,0.628292,0.927968
3,64,64,0.005,0.672404,0.936266
4,64,32,0.001,0.641190,0.935329
5,64,32,0.005,0.674935,0.940242
6,32,128,0.001,0.623451,0.928120
7,32,128,0.005,0.625341,0.927554
8,32,64,0.001,0.628471,0.927213
9,32,64,0.005,0.670402,0.935579


In [50]:
seven_trustMLP.to_csv("/content/drive/MyDrive/00.multi_emtions/02.result/00-2extra2.MLP+senti7emo_except_trust.csv",index=False)